In [2]:
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

# read in CSV data and load into vector database
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

raw_documents = CSVLoader("./data/50_books_with_metadata.csv").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

books_db = Chroma.from_documents(documents=raw_documents, embedding=embeddings)


In [4]:
# perform similarity search on vector database and return top hits
def get_recommendations(
        query: str,
        top_hits: int = 5) -> pd.DataFrame:
    results = books_db.similarity_search(query, k=top_hits)

    search_hits = []
    for i in range(0, len(results)):
        [isbn10, title, authors, categories, thumbnail, description, published_year, average_rating, num_pages, ratings_count] = results[i].page_content.split("\n")
        search_hits.append({
            'isbn10': isbn10.replace('isbn10: ', ''),
            'title': title.replace('title: ',''),
            'authors': authors.replace('authors: ',''),
            'categories': categories.replace('categories: ', ''),
            'thumbnail': thumbnail.replace('thumbnail: ', ''),
            'description': description.replace('description: ',''),
            'published_year': published_year.replace('published_year: ', ''),
            'average_rating': average_rating.replace('average_rating: ', ''),
            'num_pages': num_pages.replace('num_pages: ', ''),
            'ratings_count': ratings_count.replace('ratings_count: ', ''),
        })
    return pd.DataFrame(search_hits)

get_recommendations("A book about murder mysteries")

,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,7120966,A Murder is Announced,Agatha Christie,"Detective and mystery stories, English",http://books.google.com/books/content?id=QEEvz...,The villagers of Chipping Cleghorn are amazed ...,2002,3.98,382,680
1,7113803,Murder in Mesopotamia,Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=oFkbc...,An archaeologist's wife is murdered on the sho...,2001,3.89,264,25692
2,7119356,Appointment with Death,Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=lSYws...,A repugnant Amercian widow is killed during a ...,2001,3.86,303,24859
3,7120877,They Do it with Mirrors,Agatha Christie,American fiction,http://books.google.com/books/content?id=HUuQY...,A man is shot at in a juvenile reform home - b...,2002,3.75,284,415
4,000649045X,Witness for the Prosecution & Selected Plays,Agatha Christie,English drama,http://books.google.com/books/content?id=_9u7A...,Newly-Jacketed Edition Designed To Celebrate T...,1995,4.05,352,3908
